# G-LiHT 

This notebook illustrates search, directly loading DEMs, and downloading DEMs and LPCs for local processing

| Dataset                      | Alias      | Type      | Start      | End        | Extent         | Source                                                                      |
| ---------------------------- | ---------- | --------- | ---------- | ---------- | -------------- | --------------------------------------------------------------------------- |
| NASA G-LiHT                  | gliht      | LiDAR     | 2011-07-28 | 2020-03-12 | US Territories | [NASA G-LiHT](https://gliht.gsfc.nasa.gov/)         

In [ ]:
import coincident
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
# %config InlineBackend.figure_format = 'retina'

## Search 

We'll search for data in Maryland in July 2017

In [ ]:
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/MD/shape.geojson"
)
aoi = aoi.simplify(0.01)
m = aoi.explore(color="black")

gf = coincident.search.search(
    dataset="gliht",
    intersects=aoi,
    datetime="2017-07",
)

print(f"Found {len(gf)} G-LiHT Items")
print(f"G-LiHT collections: {gf.collection.unique()}")
gf.explore(m=m, color="magenta")

## Get Data

NASA G-LiHT has many different provided gridded datasets. The following collections below are the current datasets supported by `coincident`.

| Collection                      | Description      |
| -- | -- | 
| GLORTHO_001 | orthorectified high-resolution aerial photography |
| GLCHMT_001 | maximum canopy height and canopy variability information |
| GLDSMT_001 | Digital Surface Model, Mean, Aspect, Rugosity, and Slope |
| GLDTMT_001 | bare earth elevation, aspect and slope on the EGM96 Geoid |
| GLLIDARPC_001 | LiDAR Point Cloud data product (LAS format) |

```{note}
Not all G-LiHT flights will contain every single product listed. For example, a flight may have 'dsm' data but not 'ortho' data.
```

In [ ]:
# Subset a particular scene
collection = "GLDTMT_001"
gf_gliht = gf.query(f'collection == "{collection}"')
gf_gliht.id

In [ ]:
# Just use the first one
gs_item = gf_gliht.iloc[0]
c = gs_item.geometry.centroid
mini_aoi = gpd.GeoDataFrame(
    geometry=[box(c.x - 0.0045, c.y - 0.0045, c.x + 0.0045, c.y + 0.0045)],
    crs="EPSG:4326",
)

In [ ]:
m = gf_gliht.explore(column="id")
mini_aoi.explore(m=m, color="red", style_kwds={"fill": False, "weight": 3})

### Stream 

First we'll show how to stream a subset of a gridded DSM directly into Xarray

```{important}
Unlike the G-LiHT search, you will need NASA Earthdata credentials (aka EarthData Login (EDL)) to read in and download the gridded datasets from G-LiHT. This requires creating an [EDL Token](https://urs.earthdata.nasa.gov/documentation/for_users/user_token) and making sure you've set the environment variable `EARTHDATA_TOKEN=xxxxx`
```

In [ ]:
%%time

da = coincident.io.xarray.load_gliht(
    item=gs_item,
    aoi=mini_aoi,
)
da

In [ ]:
coincident.plot.plot_dem(da)
plt.title(gs_item.id);

### Download

#### Download multiple STAC Item Assets

This will put STAC metadata, extended metadata, a browse/thumbnail image, and a TIF in a specified folder

In [ ]:
# back to a GeoDataFrame from a Series
gfi = gpd.GeoDataFrame([gs_item])

# And to a Pystac Item
items = coincident.search.to_pystac_items(gfi)
items[0]

In [ ]:
# Download the item assets (browse, metadata, thumbnail)
local_item = await coincident.io.download.download_item(
    items[0],
    path="/tmp/gliht/",
)

#### Download a single file

In [ ]:
tif_key = [x for x in gs_item.assets.keys() if x.startswith("001")][0]
href = gs_item.assets[tif_key]["href"]
coincident.io.download.download_files([href], "/tmp/")